# Generación de features

Esta notebook calcula los features de cada concepto usando las tablas de procedimientos, participantes, scraper, proveedores sancionados, RFC apócrifos o fantasma, y montos máximos permitidos por tipo de contratación. Este código fue probado en una computadora con ubuntu 16.04 y con 16 gb de memoria RAM.



In [2]:
import os
import glob
import inspect
import unicodedata
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from itertools import product
from functools import partial
from pathlib import Path

# dask
import dask
import dask.dataframe as dd
from dask.distributed import Client

# Funciones compranet

from utils.clean_data import clean_base_rfc, clean_base_sancionados

from features.general import (
    conteo_procedimientos,
    monto_total,
    num_proveedores_unicos,
    numero_contratos
)

from features.competencia import (
    c4_monto_total,
    c4_num_procedimientos,
    contratos_por_proveedor,
    disminucion_en_participacion,
    id_por_contratos,
    id_por_monto,
    ihh_por_contratos,
    ihh_por_monto,
    importe_promedio_por_contrato,
    pc_licitaciones_con_un_participante,
    pc_monto_adj_directa_inv3,
    pc_partipaciones_promedio,
    pc_procedimientos_adj_directa_inv3,
    procs_por_participantes_unicos,
    procs_promedio_por_participantes,
    tendencia_adjudicacion_directa
)

from features.transparencia import (
    contratos_promedio_por_procedimimento,
    pc_adjudicaciones_incompletas,
    pc_inconsistencias_en_monto,
    pc_invitaciones_incompletas,
    pc_licitaciones_incompletas,
    pc_procs_con_provs_faltantes,
    pc_procs_sin_apertura,
    pc_procs_sin_archivos,
    pc_procs_sin_contrato,
    pc_procs_sin_fallo,
    pc_procs_sin_junta_aclaracion,
    porcentaje_procs_presenciales,
    promedio_datos_faltantes_por_contrato,
    promedio_procs_por_archivo,
    tendencia_no_publicacion_contratos
)


from features.anomalias import (
    participantes_por_ganadores,
    monto_con_rfc_fantasma,
    monto_con_sancionados,
    pc_adj_directas_excedieron_monto,
    pc_contratos_con_convenio,
    pc_estratificacion_mal_reportada,
    pc_invitaciones_excedieron_monto,
    pc_licitaciones_internacionales_menor_20_dias,
    pc_licitaciones_internacionales_menor_40_dias,
    pc_licitaciones_nacionales_menor_15_dias,
    pc_procs_sin_convocatoria,
    promedio_convenios_por_proc
)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline
%config IPCompleter.use_jedi = False
%load_ext autoreload
%autoreload 2

client = Client()
client

Client Scheduler: tcp://127.0.0.1:33826 Dashboard: http://127.0.0.1:8787,Cluster Workers: 8 Cores: 8 Memory: 16.69 GB


## Procedimientos

La base de datos "procedimientos.parquet" se puede obtener de [esta sección del README](https://github.com/imco/irc#datos-procesados). Si el parquet no es accesible, se puede descargar el PSV y transformarlo con la herramienta disponible en `utils/csv_to_parquet.py`.

In [4]:
df_procedimientos = pq.read_table('../data/bases/procedimientos.parquet', use_threads=True).to_pandas()

print(df_procedimientos.shape)
df_procedimientos = df_procedimientos.loc[df_procedimientos.GOBIERNO == 'APF', :]
df_procedimientos = df_procedimientos.drop('GOBIERNO', axis=1)

df_procedimientos = df_procedimientos.loc[~df_procedimientos.PROVEEDOR_CONTRATISTA.isnull()]
print(df_procedimientos.shape)
df_procedimientos = df_procedimientos.loc[df_procedimientos.CLAVEUC != 'MISSING']

# Rellenar los valores faltantes de la columna "caracter"
caracter = df_procedimientos.CARACTER.mask(df_procedimientos.CARACTER == '').fillna('SIN REPORTAR')
df_procedimientos = df_procedimientos.assign(CARACTER=caracter)

mapa_codigos_caracter = {'N': 'NACIONAL',
                         'I': 'INTERNACIONAL',
                         'T': 'INTERNACIONAL BAJO TLC'}

codigos_caracter = df_procedimientos.loc[df_procedimientos.CARACTER == 'SIN REPORTAR'].NUMERO_PROCEDIMIENTO.map(
    lambda x: x.split('-')[2][0] if isinstance(x, str) else x
)
codigos_caracter = codigos_caracter.map(lambda x: mapa_codigos_caracter.get(x, 'SIN REPORTAR'))

df_procedimientos.loc[df_procedimientos.CARACTER == 'SIN REPORTAR', 'CARACTER'] = codigos_caracter

del caracter, mapa_codigos_caracter, codigos_caracter
print(df_procedimientos.shape)
df_procedimientos.head()

(994177, 47)
(864620, 46)
(864583, 46)


,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,FECHA_ARCHIVO,CLAVEUC_REAL,IMPORTE_PESOS
70,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,NaN,MULTISERVICIOS Y ASESORIAS DEL CARIBE,HABILITADO,None,319200.0,NaN,MXN,IA-009JZL032-N1-2012,MIXTA,INVITACION A CUANDO MENOS TRES,165782,SERVICIO DE LIMPIEZA CZM 2012,nan,ADQUISICIONES,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,None,2011-12-16 00:00:00,2011-12-19 00:00:00,None,2012-01-01 00:00:00,2013-02-28 00:00:00,115807,SERVICIO DE LIMPIEZA CZM 2012,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,009JZL032,319200.0
71,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,NaN,ALEJANDRO ALARCON JIMON,HABILITADO,None,26097.0,NaN,MXN,AA-009JZL032-N4-2012,PRESENCIAL,ADJUDICACION DIRECTA,165738,MANTENIMIENTO A EXTINTORES CZM,nan,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO,2012-08-10 15:59:00,2012-08-10 16:05:00,None,2012-08-20 00:00:00,2012-08-26 00:00:00,2012-12-31 00:00:00,242370,SERVICIO DE MANTENIMIENTO A EXTINTORES AD CZM ...,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,009JZL032,26097.0
72,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,5578.0,TRANSPORTE ESPECIALIZADO DE PERSONAL NACIONAL,HABILITADO,MEDIANA,361200.0,NaN,MXN,IA-009JZL032-N2-2012,MIXTA,INVITACION A CUANDO MENOS TRES,148131,SERVICIO DE TRANSPORTE DE PERSONAL DE LA ESTAC...,nan,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,2011-12-09 14:51:00,2011-12-16 14:00:00,2011-12-19 00:00:00,None,2012-01-01 00:00:00,2013-02-28 00:00:00,116667,SERVICIO DE TRANSPORTE DE PERSONAL CZM 2012,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,009JZL032,361200.0
73,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,NaN,COMERCIALIZADORA EL MAHARAJA,HABILITADO,None,334264.0,NaN,MXN,AA-009JZL032-N5-2011,MIXTA,ADJUDICACION DIRECTA,276283,SERVICIO DE COMEDOR DEL PERSONAL DE LA ESTACIO...,nan,ADQUISICIONES,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,2011-12-28 12:45:00,2011-12-28 17:00:00,None,None,2012-01-01 00:00:00,2013-02-28 00:00:00,123933,SERVICIO DE COMEDOR AD CZM 2012,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,009JZL032,334264.0
74,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MEDIANA,NaN,COMERCIALIZADORA EL MAHARAJA,HABILITADO,None,285948.0,NaN,MXN,AA-009JZL032-N5-2011,MIXTA,ADJUDICACION DIRECTA,148083,SERVICIO DE COMEDOR PARA EL PERSONAL DE LA EST...,nan,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,NaN,NO,2011-12-28 12:45:00,2011-12-28 17:00:00,None,2011-12-30 00:00:00,2012-01-01 00:00:00,2012-12-31 00:00:00,123933,SERVICIO DE COMEDOR AD CZM 2012,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,009JZL032,285948.0


## Scraper 

Fuente: [Tabla limpia del scraper](https://s3-us-west-2.amazonaws.com/opi-compranet/public/data/tabla_scraper_features.csv) obtenida de la [sección Datos Procesados en imco/irc](https://github.com/imco/irc#datos-procesados)

In [4]:
cols_scraper = [
    'CODIGO_EXPEDIENTE',
    'archivo_anexos', 'archivo_apertura',
    'archivo_contrato', 'archivo_convocatoria',
    'archivo_fallo', 'archivo_junta',
    'numero_archivos','numero_convenios',
]

df_scraper = pd.read_csv(
    '../data/bases/tabla_scraper_features.csv',
    usecols=cols_scraper, dtype={'CODIGO_EXPEDIENTE': str}
)
# Se ordenan las columnas de la tabla
df_scraper = df_scraper.loc[:, cols_scraper]

# Join con procedimientos para obtener los tipos de contratos,
# numero de contratos, fechas de inicio, CLAVEUC, etc
cols_procs = [
    'CLAVEUC', 'NUMERO_PROCEDIMIENTO', 'CODIGO_EXPEDIENTE',
    'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION'
]

df_contratos = (df_procedimientos.groupby(cols_procs, as_index=False)
                                 .CODIGO_CONTRATO.count()
                                 .rename(columns={'CODIGO_CONTRATO': 'numero_contratos'}))
print(df_contratos.shape)
df_fechas = df_procedimientos[['CODIGO_EXPEDIENTE', 'FECHA_INICIO']]
df_fechas = df_fechas.drop_duplicates(subset=['CODIGO_EXPEDIENTE'])
df_contratos = pd.merge(df_contratos, df_fechas, on='CODIGO_EXPEDIENTE', how='inner')
print(df_contratos.shape)
df_scraper = pd.merge(df_scraper, df_contratos, on='CODIGO_EXPEDIENTE', how='inner')
del df_contratos, df_fechas, cols_procs
print(df_scraper.shape)
df_scraper.head(3)

(679938, 6)
(679938, 7)
(620189, 15)


,CODIGO_EXPEDIENTE,archivo_anexos,archivo_apertura,archivo_contrato,archivo_convocatoria,archivo_fallo,archivo_junta,numero_archivos,numero_convenios,CLAVEUC,NUMERO_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION,numero_contratos,FECHA_INICIO
0,599,0,1,0,0,1,1,58,0,018TOQ054,LO-018TOQ054-T2-2010,LICITACION PUBLICA,OBRA PUBLICA,1,2012-01-29
1,1171,0,1,1,1,1,1,6,1,041A00001,LA-010A00001-N3-2011,LICITACION PUBLICA,SERVICIOS,1,2012-01-01
2,1610,0,0,1,0,0,0,3,0,018TOQ999,IA-018TOQ999-I2-2011,INVITACION A CUANDO MENOS TRES,SERVICIOS,1,2014-03-15


## Participantes

In [5]:
cols_parts = [
    'CLAVEUC', 'NUMERO_PROCEDIMIENTO', 'CODIGO_EXPEDIENTE',
    'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION', #'FORMA_PROCEDIMIENTO',
    'PROVEEDOR_CONTRATISTA', 'ESTATUS_DE_PROPUESTA',
    'ESTATUS_FALLO', 'PRECIO_TOTAL'
]
df_participantes = dd.read_parquet('../data/bases/participantes_parquet',
                                   engine='pyarrow', columns=cols_parts)

df_participantes = df_participantes.loc[~df_participantes.PROVEEDOR_CONTRATISTA.isnull()]
df_participantes = df_participantes.loc[~df_participantes.CLAVEUC.isnull()]
df_participantes = df_participantes.assign(
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.fillna('SIN REPORTAR'),
    ESTATUS_DE_PROPUESTA=df_participantes.ESTATUS_DE_PROPUESTA.fillna('SIN REPORTAR'),
)
df_participantes = df_participantes.compute()

# Se saca el valor de TIPO_CONTRATACION del número de procedimiento
claves_procs = {
    'L': 'LICITACION PUBLICA',
    'I': 'INVITACION A CUANDO MENOS TRES',
    'A': 'ADJUDICACION DIRECTA',
    'O': 'LICITACION PUBLICA CON OSD',
    'S': 'ADJUDICACION DIRECTA',
    'P': 'PROYECTO DE CONVOCATORIA',
    'E': 'LICITACION PUBLICA',
    'X': 'SIN REPORTAR',
}
claves_contrs = {
    'A': 'ADQUISICIONES',
    'O': 'OBRA PUBLICA',
    'X': 'SIN REPORTAR'
}

claves = df_participantes.NUMERO_PROCEDIMIENTO.map(lambda x: x.split('-')[0])
procedimientos = claves.map(lambda x: claves_procs.get(x[0], None))
contrataciones = claves.map(lambda x: claves_contrs.get(x[1], None))

df_participantes = df_participantes.assign(
    TIPO_PROCEDIMIENTO=df_participantes.TIPO_PROCEDIMIENTO.mask(
        df_participantes.TIPO_PROCEDIMIENTO == 'SIN PLANTILLA', procedimientos),
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.mask(
        df_participantes.TIPO_CONTRATACION == 'SIN REPORTAR', contrataciones)
)

df_participantes = df_participantes.assign(
    TIPO_PROCEDIMIENTO=df_participantes.TIPO_PROCEDIMIENTO.fillna('SIN REPORTAR'),
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.fillna('SIN REPORTAR'),
)

del claves, procedimientos, contrataciones

# Se lee la tabla de procedimientos para obtener los tipos de proc y contratacion
col_proc = 'NUMERO_PROCEDIMIENTO'
cols_procs_tipos = [
    col_proc, 'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION']

df_procs_tipos = pq.read_table('../data/bases/procedimientos.parquet',
                               nthreads=8, columns=cols_procs_tipos).to_pandas()
 
print(df_procs_tipos.shape)
df_procs_tipos = df_procs_tipos.drop_duplicates(subset=[col_proc])
print(df_procs_tipos.shape)
# se renombra para saber de donde viene antes del merge
df_procs_tipos = df_procs_tipos.rename(
    columns={'TIPO_CONTRATACION': 'TIPO_CONTRATACION_AUX',
             'TIPO_PROCEDIMIENTO': 'TIPO_PROCEDIMIENTO_AUX'}
)

df_participantes = pd.merge(df_participantes,
                            df_procs_tipos,
                            on='NUMERO_PROCEDIMIENTO',
                            how='left')

del df_procs_tipos, claves_contrs, claves_procs, cols_parts, cols_procs_tipos
# se cambian los valores en caso de que no se tengan disponibles
# usando los de procedimientos
df_participantes = df_participantes.assign(
    TIPO_PROCEDIMIENTO=df_participantes.TIPO_PROCEDIMIENTO.mask(
        df_participantes.TIPO_PROCEDIMIENTO == 'SIN REPORTAR',
        df_participantes.TIPO_PROCEDIMIENTO_AUX).fillna('SIN REPORTAR'),
    TIPO_CONTRATACION=df_participantes.TIPO_CONTRATACION.mask(
        df_participantes.TIPO_CONTRATACION == 'SIN REPORTAR',
        df_participantes.TIPO_CONTRATACION_AUX).fillna('SIN REPORTAR'),
)

df_participantes = df_participantes.drop(['TIPO_PROCEDIMIENTO_AUX',
                                          'TIPO_CONTRATACION_AUX'], axis=1)

# Se cambia el valor al normalizado

df_participantes.loc[
    df_participantes.TIPO_PROCEDIMIENTO == 'INVITACION A CUANDO MENOS TRES', 'TIPO_PROCEDIMIENTO'
] = 'INVITACION A CUANDO MENOS TRES'

df_participantes.loc[
    df_participantes.TIPO_CONTRATACION == 'SERVICIO DE OBRA PUBLICA', 'TIPO_CONTRATACION'
] = 'SERVICIOS RELACIONADOS CON LA OP'

print(df_participantes.shape)
df_participantes.head()

(994177, 3)
(789837, 3)
(5216946, 9)


,CLAVEUC,NUMERO_PROCEDIMIENTO,CODIGO_EXPEDIENTE,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION,PROVEEDOR_CONTRATISTA,ESTATUS_DE_PROPUESTA,ESTATUS_FALLO,PRECIO_TOTAL
0,009000972,IO-009000972-N30-2011,93548,INVITACION A CUANDO MENOS TRES,SERVICIOS RELACIONADOS CON LA OP,CONSTRUCCIONES Y PAVIMENTOS BRUFA,GANADOR,GANADOR,8963339.44
1,009000972,IO-009000972-N30-2011,93548,INVITACION A CUANDO MENOS TRES,SERVICIOS RELACIONADOS CON LA OP,CONSORCIO CONSTRUCTOR LOBO,PERDEDOR,GANADOR,10253719.14
2,009000972,IO-009000972-N30-2011,93548,INVITACION A CUANDO MENOS TRES,SERVICIOS RELACIONADOS CON LA OP,CONSTRUCCIONES CESAR VALENZUELA VELASCO,PERDEDOR,GANADOR,10104415.88
3,006HHG001,LA-006HHG001-N64-2011,97983,LICITACION PUBLICA,ADQUISICIONES,ASESORIA EN COMPUTACION SERVICIOS Y ELECTRONICA,GANADOR,GANADOR,1970.00
4,018T4I008,AA-018T4I008-N8-2012,135592,ADJUDICACION DIRECTA,ADQUISICIONES,MEDICAL DEPOT,PERDEDOR,GANADOR,9343.80


### RFC y Sancionados

In [6]:
# RFC fantasma
df_fantasma = pd.read_csv(
    '../data/bases/RFC_fantasma.csv', parse_dates=['Publicación página SAT definitivos'],
    usecols=['RFC', 'Nombre del Contribuyente', 'Publicación página SAT definitivos'],
    dtype={'RFC': str, 'Nombre del Contribuyente': str}, encoding='iso-8859-1', skiprows=2
)
df_fantasma = df_fantasma.rename(columns={'Nombre del Contribuyente': 'Nombre del Contribuyente'.upper()})
df_fantasma = df_fantasma.loc[df_fantasma.RFC != 'XXXXXXXXXXXX']
df_fantasma = clean_base_rfc(df_fantasma)

# Proveedores sancionados
df_sancionados = pd.read_excel('../data/bases/SancionProveedoresContratistas.xls',
                               dtype={'Expediente': str})
df_sancionados = clean_base_sancionados(df_sancionados)
# Hay un registro con expediente vacío, de la tabla de procedimientos se observa
# que los contratos los realizó en 2014
df_sancionados.loc[
    df_sancionados.PROVEEDOR_CONTRATISTA == 'ALBA MARIA DE LA ASUNCION HERRASTI FERNANDEZ', 'Expediente'
] = 'XXXX/2014'
df_sancionados = df_sancionados.assign(
    Year=df_sancionados.Expediente.map(
        lambda x: int(x.split('/')[1])
    )
)


### Montos máximos

In [7]:
df_montos_maximos = pd.read_csv('../data/bases/Montos_maximos.csv',
                                usecols=['Año', 'Tipo de contratación', ' Adjudicación directa ', ' INV3 '],
                                dtype={'Año': int},
                                encoding='iso-8859-1')
df_montos_maximos = df_montos_maximos.rename(columns={c: c.strip() for c in df_montos_maximos.columns})
df_montos_maximos.loc[:, 'INV3'] = (df_montos_maximos.INV3.str.strip()
                                                          .str.replace(',', '')
                                                          .astype(int))
df_montos_maximos.loc[:, 'Adjudicación directa'] = (df_montos_maximos['Adjudicación directa'].str.strip()
                                                                                             .str.replace(',', '')
                                                                                             .astype(int))
df_montos_maximos.loc[:, 'Tipo de contratación'] = (df_montos_maximos['Tipo de contratación'].str.normalize('NFD')
                                                                                             .str.encode('ascii', 'ignore')
                                                                                             .str.decode('utf-8')
                                                                                             .str.upper())
print(df_montos_maximos.shape)


(30, 4)


## Cálculo de features

En esta parte debes de cambiar las direcciones de salida de los features, nosotros usamos: '../data/conceptos/{concepto}/{tipo de contratacion}/features.csv'

In [8]:
conceptos = ['general', 'competencia', 'transparencia', 'anomalias']

tipos_contratacion = (
    'ADQUISICIONES',
    'SERVICIOS',
    'OBRA PUBLICA',
    'ARRENDAMIENTOS',
    'SERVICIOS RELACIONADOS CON LA OP'
)


for tipo in tipos_contratacion:
    print('*' * 50)
    print(tipo)
    # sub-tablas por tipo
    df_procs_aux = df_procedimientos.loc[
        (df_procedimientos.TIPO_CONTRATACION == tipo)
    ]
    df_scraper_aux = df_scraper.loc[
        (df_scraper.TIPO_CONTRATACION == tipo)
    ]
    df_parts_aux = df_participantes.loc[
        (df_participantes.TIPO_CONTRATACION == tipo)
    ]
    df_base = pd.DataFrame(df_procs_aux.CLAVEUC.unique(),
                           columns=['CLAVEUC']).set_index('CLAVEUC')
    
    # - general ---------------------------------
    dfs_general = [
        conteo_procedimientos(df_procs_aux),
        monto_total(df_procs_aux),
        num_proveedores_unicos(df_procs_aux),
        numero_contratos(df_procs_aux)
    ]
    dfs_general = [
        df.set_index('CLAVEUC')
        for df in dfs_general
        if df is not None
    ]
    print('General', [df.shape[0] for df in dfs_general])
    df_general = pd.concat(dfs_general, axis=1, join='outer')
    df_general = df_base.join(df_general, how='left')
    df_general.to_csv(f'../data/conceptos/general/{tipo}/features.csv',
                      index=True, quoting=1, encoding='utf-8')
    print(f'shape general: {df_general.shape}')
    del dfs_general, df_general
    print('-' * 50)
    
    # - competencia ---------------------
    dfs_competencia = [
        c4_monto_total(df_procs_aux),
        c4_num_procedimientos(df_procs_aux),
        contratos_por_proveedor(df_procs_aux),
        disminucion_en_participacion(df_parts_aux),
        id_por_contratos(df_procs_aux),
        id_por_monto(df_procs_aux),
        ihh_por_contratos(df_procs_aux),
        ihh_por_monto(df_procs_aux),
        importe_promedio_por_contrato(df_procs_aux),
        pc_licitaciones_con_un_participante(df_parts_aux),
        pc_monto_adj_directa_inv3(df_procs_aux),
        pc_partipaciones_promedio(df_parts_aux),
        pc_procedimientos_adj_directa_inv3(df_procs_aux),
        procs_por_participantes_unicos(df_parts_aux),
        procs_promedio_por_participantes(df_parts_aux),
        tendencia_adjudicacion_directa(df_procs_aux)
    ]
    dfs_competencia = [
        df.set_index('CLAVEUC')
        for df in dfs_competencia
        if df is not None
    ]
    print('Competencia', [df.shape[0] for df in dfs_competencia])
    df_competencia = pd.concat(dfs_competencia, axis=1, join='outer')
    df_competencia = df_base.join(df_competencia, how='left')
    print(f'shape competencia: {df_competencia.shape}')
    df_competencia.to_csv(f'../data/conceptos/competencia/{tipo}/features.csv',
                          index=True, quoting=1, encoding='utf-8')
    del df_competencia, dfs_competencia
    print('-' * 50)
    # - transparencia ----------------------------------
    dfs_transparencia = [
        contratos_promedio_por_procedimimento(df_procs_aux),
        pc_adjudicaciones_incompletas(df_scraper_aux),
        pc_inconsistencias_en_monto(df_procs_aux, df_parts_aux),
        pc_invitaciones_incompletas(df_scraper_aux),
        pc_licitaciones_incompletas(df_scraper_aux),
        pc_procs_con_provs_faltantes(df_procs_aux, df_parts_aux),
        pc_procs_sin_apertura(df_scraper_aux),
        pc_procs_sin_archivos(df_scraper_aux),
        pc_procs_sin_contrato(df_scraper_aux),
        pc_procs_sin_fallo(df_scraper_aux),
        pc_procs_sin_junta_aclaracion(df_scraper_aux),
        porcentaje_procs_presenciales(df_procs_aux),
        promedio_datos_faltantes_por_contrato(df_procs_aux),
        promedio_procs_por_archivo(df_scraper_aux),
        tendencia_no_publicacion_contratos(df_scraper_aux),
    ]
    dfs_transparencia = [
        df.set_index('CLAVEUC')
        for df in dfs_transparencia
        if df is not None
    ]
    print('Transparencia', [df.shape[0] for df in dfs_transparencia])
    df_transparencia = pd.concat(dfs_transparencia, axis=1, join='outer')
    df_transparencia = df_base.join(df_transparencia, how='left')
    print(f'shape transparencia: {df_transparencia.shape}')
    df_transparencia.to_csv(f'../data/conceptos/transparencia/{tipo}/features.csv',
                            index=True, quoting=1, encoding='utf-8')
    del df_transparencia, dfs_transparencia
    print('-' * 50)
    # - anomalias -
    dfs_anomalias = [
        participantes_por_ganadores(df_parts_aux),
        monto_con_rfc_fantasma(df_procs_aux, df_fantasma),
        monto_con_sancionados(df_procs_aux, df_sancionados),
        pc_adj_directas_excedieron_monto(df_procs_aux, df_montos_maximos, tipo_contratacion=tipo),
        pc_contratos_con_convenio(df_procs_aux),
        pc_estratificacion_mal_reportada(df_procs_aux),
        pc_invitaciones_excedieron_monto(df_procs_aux, df_montos_maximos, tipo_contratacion=tipo),
        pc_licitaciones_internacionales_menor_20_dias(df_procs_aux),
        pc_licitaciones_internacionales_menor_40_dias(df_procs_aux),
        pc_licitaciones_nacionales_menor_15_dias(df_procs_aux),
        pc_procs_sin_convocatoria(df_scraper_aux),
        promedio_convenios_por_proc(df_scraper_aux)
    ]
    dfs_anomalias = [
        df.set_index('CLAVEUC')
        for df in dfs_anomalias
        if df is not None
    ]
    print('Anomalias', [df.shape[0] for df in dfs_anomalias])
    df_anomalias = pd.concat(dfs_anomalias, axis=1, join='outer')
    df_anomalias = df_base.join(df_anomalias, how='left')
    print(f'shape anomalias: {df_anomalias.shape}')
    df_anomalias.to_csv(f'../data/conceptos/anomalias/{tipo}/features.csv',
                        index=True, quoting=1, encoding='utf-8')
    del df_anomalias, dfs_anomalias
    print('-' * 50)

**************************************************
ADQUISICIONES
General [1435, 1435, 1435, 1435]
shape general: (1435, 4)
--------------------------------------------------
Competencia [1435, 1435, 1435, 1756, 1435, 1435, 1435, 1435, 1435, 1756, 1435, 1756, 1435, 1756, 1756, 1435]
shape competencia: (1435, 16)
--------------------------------------------------
Transparencia [1435, 1409, 1435, 1409, 1409, 1435, 1409, 1409, 1409, 1409, 1409, 1435, 1435, 1409, 1409]
shape transparencia: (1435, 15)
--------------------------------------------------
Anomalias [1550, 1435, 1435, 1435, 1435, 1435, 1435, 1435, 1435, 1435, 1409, 1409]
shape anomalias: (1435, 12)
--------------------------------------------------
**************************************************
SERVICIOS
General [1548, 1548, 1548, 1548]
shape general: (1548, 4)
--------------------------------------------------
Competencia [1548, 1548, 1548, 1481, 1548, 1548, 1548, 1548, 1548, 1481, 1548, 1481, 1548, 1481, 1481, 1548]
shape c

## (Opcional)  Convertir tabla de participantes a parquet

En esta parte se convierte a parquet las tablas de participantes, es más rápido leerlas en este formato

In [24]:

cols_parts = [
    'CLAVEUC', 'NUMERO_PROCEDIMIENTO', 'CODIGO_EXPEDIENTE', 
    'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION', 'FORMA_PROCEDIMIENTO',
    'PROVEEDOR_CONTRATISTA',
    'ESTATUS_DE_PROPUESTA', 'ESTATUS_PARTIDA', 'ESTATUS_FALLO',
    'PRECIO_TOTAL',
]

df_participantes = dd.read_csv(
    '../data/processed/participantes/participantes_*_*.psv', sep='|', usecols=cols_parts,
    dtype={
        'TIPO_PROCEDIMIENTO': str,
        'TIPO_CONTRATACION': str,
        'FORMA_PROCEDIMIENTO': str,
        'ESTATUS_DE_PROPUESTA': str,
        'PERIODO': str,
        'DESCRIPCION_CUCOP': str,
        'PROVEEDOR_CONTRATISTA': str,
        'CODIGO_EXPEDIENTE': str
    }
)
# se ordenan las columnas
df_participantes = df_participantes.loc[:, cols_parts]
df_participantes.to_parquet('../data/bases/participantes_parquet/', compression='snappy')
df_test = dd.read_parquet('../data/bases/participantes_parquet/', engine='pyarrow')
df_test.head()

## (Opcional) Pegar nombres de uc y dependencias

In [6]:
directorio_uc_path = '../data/bases/nombres_unidades_compradoras.csv'
df_directorio = pd.read_csv(directorio_uc_path, dtype={'CLAVEUC': str})
nombres_dep = {row.CLAVEUC: row.DEPENDENCIA
               for row in df_directorio.itertuples()}
nombres_uc = {row.CLAVEUC: row.NOMBRE_UC
              for row in df_directorio.itertuples()}
del df_directorio

conceptos = ['general', 'competencia', 'transparencia', 'anomalias']

tipos_contratacion = (
    'ADQUISICIONES',
    'SERVICIOS',
    'OBRA PUBLICA',
    'ARRENDAMIENTOS',
    'SERVICIOS RELACIONADOS CON LA OP'
)

for concepto in conceptos:
    for tipo in tipos_contratacion:
        df_feature = pd.read_csv(f'../data/conceptos/{concepto}/{tipo}/features.csv',
                                 dtype={'CLAVEUC': str})
        dependencia = df_feature.CLAVEUC.map(nombres_dep)
        unidad_compradora = df_feature.CLAVEUC.map(nombres_uc)
        df_feature = df_feature.assign(dependencia=dependencia,
                                       unidad_compradora=unidad_compradora)
        df_feature.to_csv(f'../data/conceptos_con_nombres/{concepto}/{tipo}/features.csv',
                          index=False, encoding='utf-8', quoting=1)